# Q-learning

In [1]:
import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: Traceback (most recent call last):
  File "/home/s1448512/miniconda3/envs/gan/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/home/s1448512/miniconda3/envs/gan/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/home/s1448512/miniconda3/envs/gan/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/home/s1448512/miniconda3/envs/gan/lib/python3.5/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/home/s1448512/miniconda3/envs/gan/lib/python3.5/imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: libcudnn.so.6: cannot open shared object file: No such fi

In [ ]:
env = gym.make('FrozenLake-v0')

In [3]:
#Initialize table with all zeros
Q = np.zeros([env.observation_space.n,env.action_space.n])
# Set learning parameters
lr = .8
y = .95
num_episodes = 2000
#create lists to contain total rewards and steps per episode
#jList = []
rList = []
for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        #Choose an action by greedily (with noise) picking from Q table
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)

In [4]:
print(Q)
env.render()

[[  2.46275963e-01   4.96566659e-03   6.73802258e-03   3.73773110e-03]
 [  1.37623641e-03   1.61911896e-04   7.31392641e-04   3.08898467e-01]
 [  2.68956811e-03   1.40111871e-01   2.48245217e-03   2.27124820e-03]
 [  2.12067620e-03   3.76357647e-04   9.18763761e-05   4.00243310e-02]
 [  2.70891424e-01   4.69234971e-04   9.14905058e-04   4.06666946e-05]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.85643404e-04   3.61416750e-04   6.30383990e-02   2.67373486e-09]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.06571144e-04   5.08542704e-04   2.20980311e-04   3.12711768e-01]
 [  8.74878435e-04   6.62441577e-01   9.56690106e-04   1.13769145e-03]
 [  5.98284626e-01   2.44145067e-04   2.90416001e-04   1.04092434e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.11396194e-04   4.21040852e-05   8.85254089e-01   2.21568931e-04]
 [  0.

In [5]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.4375


In [6]:
s = env.reset()
env.render()

d = False
              
while d == False:
    a = np.argmax(Q[s,:]) 
    s1,r,d,_ = env.step(a)
    s = s1
    # env.render()


SFFF
FHFH
FFFH
HFFG


# Q-learning with NN

In [7]:
tf.reset_default_graph()

In [8]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,16],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([16,4],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [9]:
init = tf.global_variables_initializer()

In [10]:
# Set learning parameters
y = .99
e = 0.1
num_episodes = 2000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Network
        while j < 99:
            j+=1
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:np.identity(16)[s:s+1]})
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample()
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:np.identity(16)[s1:s1+1]})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:np.identity(16)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)
print("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")


Percent of succesful episodes: 0.0935%


In [11]:
s = env.reset()
env.render()

d = False

print(Q)

while d == False:
    a = np.argmax(Q[s,:]) 
    s1,r,d,_ = env.step(a)
    s = s1
    env.render()


SFFF
FHFH
FFFH
HFFG
[[  2.46275963e-01   4.96566659e-03   6.73802258e-03   3.73773110e-03]
 [  1.37623641e-03   1.61911896e-04   7.31392641e-04   3.08898467e-01]
 [  2.68956811e-03   1.40111871e-01   2.48245217e-03   2.27124820e-03]
 [  2.12067620e-03   3.76357647e-04   9.18763761e-05   4.00243310e-02]
 [  2.70891424e-01   4.69234971e-04   9.14905058e-04   4.06666946e-05]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.85643404e-04   3.61416750e-04   6.30383990e-02   2.67373486e-09]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.06571144e-04   5.08542704e-04   2.20980311e-04   3.12711768e-01]
 [  8.74878435e-04   6.62441577e-01   9.56690106e-04   1.13769145e-03]
 [  5.98284626e-01   2.44145067e-04   2.90416001e-04   1.04092434e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.11396194e-04   4.21040852e-05   8.85254089e-01   2

# Policy gradients

In [12]:
#List out our bandits. Currently bandit 4 (index#3) is set to most often provide a positive reward.
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    #Get a random number.
    result = np.random.randn(1)
    if result > bandit:
        #return a positive reward.
        return 1
    else:
        #return a negative reward.
        return -1

In [13]:
tf.reset_default_graph()

#These two lines established the feed-forward part of the network. This does the actual choosing.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#The next six lines establish the training proceedure. We feed the reward and chosen action into the network
#to compute the loss, and use it to update the network.
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [14]:
total_episodes = 1000 #Set total number of episodes to train agent on.
total_reward = np.zeros(num_bandits) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

init = tf.initialize_all_variables()

# Launch the tensorflow graph
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        #Choose either a random action or one from our network.
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.
        
        #Update the network.
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Update our running tally of scores.
        total_reward[action] += reward
        if i % 50 == 0:
            print("Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
        i+=1
    
print("The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...and it was right!")
else:
    print("...and it was wrong!")
    

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[2017-11-02 14:14:19,549] From /Users/gvsi/.virtualenvs/gan/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:175: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


Running reward for the 4 bandits: [-1.  0.  0.  0.]
Running reward for the 4 bandits: [ -1.  19.   0.   1.]
Running reward for the 4 bandits: [ -1.  22.  -1.   1.]
Running reward for the 4 bandits: [ -2.  30.  -1.   2.]
Running reward for the 4 bandits: [ -2.  31.  -2.   6.]
Running reward for the 4 bandits: [ -2.  34.  -3.   8.]
Running reward for the 4 bandits: [ -1.  35.  -1.  10.]
Running reward for the 4 bandits: [ -2.  39.  -1.  11.]
Running reward for the 4 bandits: [ -2.  40.  -1.  12.]
Running reward for the 4 bandits: [ -2.  42.   0.  15.]
Running reward for the 4 bandits: [ -3.  38.  -1.  15.]
Running reward for the 4 bandits: [ -3.  36.  -2.  16.]
Running reward for the 4 bandits: [ -4.  39.  -1.  17.]
Running reward for the 4 bandits: [ -4.  29.  -1.  19.]
Running reward for the 4 bandits: [ -5.  34.  -1.  21.]
Running reward for the 4 bandits: [ -5.  34.  -1.  23.]
Running reward for the 4 bandits: [ -6.  29.   0.  24.]
Running reward for the 4 bandits: [ -4.  38.   1.  2